In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import warnings
import random
from sklearn.cross_validation import KFold, cross_val_score, train_test_split
from sklearn import metrics
import statsmodels.api as sm
import scipy.stats as stats 
from sklearn.linear_model import LogisticRegression , LinearRegression

from datetime import datetime
random.seed(datetime.now())
warnings.filterwarnings('ignore')

# Make plots larger
plt.rcParams['figure.figsize'] = (10, 6)

import os.path
userhome = os.path.expanduser('~')
csvfile = os.path.join(userhome, 'Desktop', 'final', 'Dengue_Daily_EN.csv')
df = pd.read_csv(csvfile)

/Users/pipiterry/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/pipiterry/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df.drop(df.columns[[1,7,8,14,15,16,17,19,26,27,28,29]], axis=1, inplace=True)
df.head()

,Date_Onset,Date_Notification,Sex,Age_Group,County_living,Township_living,Enumeration_unit,Enumeration_unit_long,Enumeration_unit_lat,First_level_dissemination_unit,Second_level_dissemination_unit,Imported,Number_of_confirmed_cases,Serotype,MOI_County_living_Code,MOI_Township_living_Code,MOI_County_living,MOI_Township_living
0,1998/01/02,1998/01/07,M,40-44,Pingtung County,Pingtung City,A1320-0136-00,120.50588,22.46425,A1320-04-008,A1320-04,N,1,NaN,10013.0,1001301.0,Pingtung County,Pingtung City
1,1998/01/03,1998/01/14,M,30-34,Pingtung County,Donggang Township,A1303-0150-00,120.45364,22.46639,A1303-09-007,A1303-09,Y,1,DENV2,10013.0,1001303.0,Pingtung County,Donggang Township
2,1998/01/13,1998/02/18,M,55-59,Yilan County,Yilan City,A0201-0449-00,121.75143,24.74922,A0201-23-006,A0201-23,Y,1,NaN,10002.0,1000201.0,Yilan County,Yilan City
3,1998/01/15,1998/01/23,M,35-39,Kaohsiung City,Lingya Dist.,A6408-0153-00,120.33814,22.63032,A6408-10-010,A6408-10,N,1,NaN,64.0,6400800.0,Kaohsiung City,Lingya District
4,1998/01/20,1998/02/04,M,55-59,Yilan County,Wujie Township,A0209-0232-00,121.79826,24.68457,A0209-10-005,A0209-10,N,1,NaN,10002.0,1000209.0,Yilan County,Wujie Township


In [3]:
#這裡開始是把文字描述變成數字去呈現
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
le=preprocessing.LabelEncoder()
le.fit(df.Age_Group)
list(le.classes_)
df['Age_Group'].replace('0', '0-9', inplace=True)
df['Age_Group'].replace('1', '0-9', inplace=True)
df['Age_Group'].replace('2', '0-9', inplace=True)
df['Age_Group'].replace('3', '0-9', inplace=True)
df['Age_Group'].replace('4', '0-9', inplace=True)
df['Age_Group'].replace('5-9', '0-9', inplace=True)
le.fit(df.Age_Group)
list(le.classes_)

['0-9',
 '10-14',
 '15-19',
 '20-24',
 '25-29',
 '30-34',
 '35-39',
 '40-44',
 '45-49',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70+']

In [4]:
df.Age_Group=le.transform(df.Age_Group)

In [5]:
#清掉空資料的情況，U表示有不知道
df['Sex'].replace('U', np.nan, inplace=True)
df.dropna(subset=['Sex'], inplace=True)
le.fit(df.Sex)
list(le.classes_)
df.Sex=le.transform(df.Sex)

In [6]:
#all
df.isnull().any()

Date_Onset                         False
Date_Notification                  False
Sex                                False
Age_Group                          False
County_living                      False
Township_living                    False
Enumeration_unit                    True
Enumeration_unit_long               True
Enumeration_unit_lat                True
First_level_dissemination_unit      True
Second_level_dissemination_unit     True
Imported                           False
Number_of_confirmed_cases          False
Serotype                            True
MOI_County_living_Code              True
MOI_Township_living_Code            True
MOI_County_living                   True
MOI_Township_living                 True
dtype: bool

In [7]:
#清掉空資料的情況，true表示有空集合
df['Serotype'].replace('', np.nan, inplace=True)
df.dropna(subset=['Serotype'], inplace=True)
df.isnull().any()

Date_Onset                         False
Date_Notification                  False
Sex                                False
Age_Group                          False
County_living                      False
Township_living                    False
Enumeration_unit                    True
Enumeration_unit_long               True
Enumeration_unit_lat                True
First_level_dissemination_unit      True
Second_level_dissemination_unit     True
Imported                           False
Number_of_confirmed_cases          False
Serotype                           False
MOI_County_living_Code              True
MOI_Township_living_Code            True
MOI_County_living                   True
MOI_Township_living                 True
dtype: bool

In [8]:
df['Enumeration_unit_long'].replace('', np.nan, inplace=True)
df.dropna(subset=['Enumeration_unit_long'], inplace=True)
df.isnull().any()

Date_Onset                         False
Date_Notification                  False
Sex                                False
Age_Group                          False
County_living                      False
Township_living                    False
Enumeration_unit                   False
Enumeration_unit_long              False
Enumeration_unit_lat               False
First_level_dissemination_unit     False
Second_level_dissemination_unit    False
Imported                           False
Number_of_confirmed_cases          False
Serotype                           False
MOI_County_living_Code              True
MOI_Township_living_Code            True
MOI_County_living                   True
MOI_Township_living                 True
dtype: bool

In [9]:
from sklearn import preprocessing
from sklearn.svm import SVC
model = SVC()
X = np.array(df[['Age_Group','Sex','Enumeration_unit_long','Enumeration_unit_lat']])
# calculate mean  
X_mean = X.mean(axis=0)  
# calculate variance   
X_std = X.std(axis=0)  
# standardize X  
X1 = (X-X_mean)/X_std  # 自己计算
# use function preprocessing.scale to standardize X  
X_scale = preprocessing.scale(X)  # 调用sklearn包的方法

Y = np.array(df['Serotype'])
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.3, random_state=0)
X_train.shape,y_train.shape

((6048, 4), (6048,))

In [10]:
#使用Support Vector Classifier來建立模型
model.fit(X_train,y_train)
predictions = model.predict(X_test)

#載入classification_report & confusion_matrix來評估模型好壞
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[ 415  301    0    0]
 [ 110 1168    0    0]
 [  42  406    0    0]
 [  98   53    0    0]]


             precision    recall  f1-score   support

      DENV1       0.62      0.58      0.60       716
      DENV2       0.61      0.91      0.73      1278
      DENV3       0.00      0.00      0.00       448
      DENV4       0.00      0.00      0.00       151

avg / total       0.47      0.61      0.53      2593



In [11]:
model.score(X_test, y_test)

0.61048978017740074

In [12]:
model.predict(X_test)

array(['DENV1', 'DENV1', 'DENV2', ..., 'DENV1', 'DENV2', 'DENV2'], dtype=object)

In [13]:
y_test

array(['DENV1', 'DENV1', 'DENV3', ..., 'DENV4', 'DENV2', 'DENV2'], dtype=object)

In [14]:
le.fit(df.Sex)
list(le.classes_)

[0, 1]

In [15]:
df.Sex=le.transform(df.Sex)

In [16]:
X = np.array(df[['Age_Group','Sex', 'Enumeration_unit_long']])
# calculate mean  
X_mean = X.mean(axis=0)  
# calculate variance   
X_std = X.std(axis=0)  
# standardize X  
X1 = (X-X_mean)/X_std  # 自己计算
# use function preprocessing.scale to standardize X  
X_scale = preprocessing.scale(X)  # 调用sklearn包的方法

Y = np.array(df['Serotype'])
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.3, random_state=0)
X_train.shape,y_train.shape

((6048, 3), (6048,))

In [17]:
#使用Support Vector Classifier來建立模型
model.fit(X_train,y_train)
predictions = model.predict(X_test)

#載入classification_report & confusion_matrix來評估模型好壞
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[ 387  329    0    0]
 [  80 1198    0    0]
 [  47  401    0    0]
 [  90   61    0    0]]


             precision    recall  f1-score   support

      DENV1       0.64      0.54      0.59       716
      DENV2       0.60      0.94      0.73      1278
      DENV3       0.00      0.00      0.00       448
      DENV4       0.00      0.00      0.00       151

avg / total       0.47      0.61      0.52      2593



In [18]:
model.score(X_test, y_test)

0.61126108754338604